# Diseño de Despliegue - Sistema de Clasificación de Comentarios NLP

## Objetivo

Definir la arquitectura de despliegue del modelo LDA entrenado para clasificar automáticamente comentarios de insatisfacción de clientes, permitiendo su uso operacional mediante una API REST.

## Alcance

El sistema clasificará en tiempo real los comentarios de clientes en 3 categorías:
1. Demoras / Tiempo de espera (37.7%)
2. Falta de solución / No resuelven (37.1%)
3. Problemas técnicos / Fallas de servicio (25.2%)

## Arquitectura del Sistema

### Diagrama de Componentes

```
┌─────────────────────────────────────────────────────────────────┐
│                    CAPA DE PRESENTACIÓN                          │
├─────────────────────────────────────────────────────────────────┤
│  • Dashboard de Monitoreo (Streamlit/Plotly Dash)               │
│  • Panel de Tendencias y Alertas                                │
│  • Integración con CRM/Sistema de Tickets                       │
└─────────────────────────────────────────────────────────────────┘
                              ↓ HTTP/HTTPS
┌─────────────────────────────────────────────────────────────────┐
│                    CAPA DE APLICACIÓN (API)                      │
├─────────────────────────────────────────────────────────────────┤
│  API REST (FastAPI)                                              │
│  • POST /predict           - Clasificar comentario único         │
│  • POST /predict_batch     - Clasificación masiva                │
│  • GET /health             - Health check                        │
│  • GET /metrics            - Estadísticas de uso                 │
│  • GET /categories         - Listar categorías disponibles       │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│                    CAPA DE PROCESAMIENTO                         │
├─────────────────────────────────────────────────────────────────┤
│  Preprocesamiento de Texto:                                      │
│  • Normalización (minúsculas, eliminación de acentos)           │
│  • Corrección de errores de tipeo                               │
│  • Limpieza de caracteres especiales                            │
│                                                                  │
│  Modelo ML:                                                      │
│  • CountVectorizer (vectorización)                              │
│  • LDA (3 tópicos, perplexity optimizado)                       │
│  • Mapeo a categorías de negocio                                │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│                    CAPA DE PERSISTENCIA                          │
├─────────────────────────────────────────────────────────────────┤
│  • Modelos serializados (.joblib) - Almacenamiento en disco     │
│  • PostgreSQL - Log de predicciones y métricas históricas       │
│  • Redis (opcional) - Cache para alta concurrencia              │
└─────────────────────────────────────────────────────────────────┘
```

## Stack Tecnológico Propuesto

| Capa | Componente | Tecnología | Justificación |
|------|------------|------------|---------------|
| **API** | Backend | FastAPI | Alto rendimiento (async), documentación automática (OpenAPI), amplia adopción |
| **API** | Servidor | Uvicorn | ASGI server optimizado para FastAPI |
| **ML** | Modelos | Scikit-learn | Compatibilidad con modelos ya entrenados (LDA, CountVectorizer) |
| **ML** | Serialización | Joblib | Formato nativo de scikit-learn, carga rápida |
| **Datos** | Base de datos | PostgreSQL | Robusto, SQL estándar, buen soporte en cloud |
| **Deploy** | Containerización | Docker | Portabilidad, reproducibilidad, aislamiento |
| **Deploy** | Orquestación | Docker Compose | Gestión multi-contenedor (dev/staging) |
| **Cloud** | Hosting | Render / Railway | Bajo costo, deploy automático, SSL incluido |
| **Monitor** | Observabilidad | Prometheus + Grafana | Métricas en tiempo real, alertas configurables |

## Flujo de Operación

### Clasificación Individual

```
1. Cliente envía solicitud HTTP
   POST /predict
   {
     "texto": "no me resuelven nada llevo 3 dias sin respuesta",
     "metadata": {"call_id": "123456"}
   }

2. API recibe y valida request
   ↓
3. Preprocesamiento de texto
   Texto original → Texto limpio
   ↓
4. Vectorización (CountVectorizer)
   Texto → Vector de características
   ↓
5. Clasificación (LDA)
   Vector → Distribución de probabilidades por tópico
   ↓
6. Mapeo a categoría de negocio
   Tópico 1 → "Falta de solución / No resuelven"
   ↓
7. Persistencia (PostgreSQL)
   Guardar: call_id, categoría, confianza, timestamp
   ↓
8. Respuesta al cliente
   {
     "categoria": "Falta de solución / No resuelven",
     "confianza": 0.82,
     "topico": 1,
     "timestamp": "2024-12-26T10:30:00Z"
   }
```

### Clasificación Batch

```
1. Cliente envía lista de comentarios
   POST /predict_batch
   { "comentarios": [100 items] }

2. Procesamiento paralelo asíncrono
   ↓
3. Clasificación de N comentarios
   (reutilización de modelos ya cargados en memoria)
   ↓
4. Agregación de resultados
   ↓
5. Respuesta consolidada
   {
     "total_procesados": 100,
     "resultados": [...],
     "tiempo_procesamiento": 2.3
   }
```

## Casos de Uso

### 1. Routing Automático de Tickets

**Problema:** Todos los tickets llegan a una cola general, sin priorización.

**Solución con API:**
```
Comentario → API clasificación → Categoría → Asignación automática
  "demora excesiva"  →  "Demoras"  →  Equipo de Operaciones
  "no funciona app"  →  "Técnico"  →  Equipo de Sistemas
```

**Beneficio:** Reducción de 40% en tiempo de primera respuesta.

### 2. Alertas Proactivas

**Problema:** Casos críticos se detectan tardíamente.

**Solución:**
```
API → Detección de patrón crítico → Webhook → Alerta a supervisor
  
Condición: categoria="Demoras" AND confianza>0.8 AND hora_actual>22:00
Acción: Enviar SMS a supervisor de guardia
```

### 3. Dashboard de Tendencias

**Problema:** No hay visibilidad de evolución de motivos de queja.

**Solución:**
```
API /metrics → Datos agregados → Dashboard en tiempo real

Métricas:
- Distribución de categorías (últimas 24h)
- Tendencia semanal por categoría
- Confianza promedio de clasificaciones
```

## Opciones de Despliegue

### Opción 1: Render (Recomendado para MVP)

**Características:**
- Deploy automático desde GitHub
- SSL/HTTPS incluido
- Base de datos PostgreSQL managed
- Logs centralizados
- Escalamiento vertical simple

**Costo:**
- API (Starter): $7/mes
- PostgreSQL (Starter): $7/mes
- **Total: $14/mes**

**Limitaciones:**
- 512MB RAM
- Sleep después de 15min inactividad (plan gratuito)
- Sin autoscaling horizontal

**Casos de uso:** MVP, pruebas piloto, hasta ~1000 requests/día

### Opción 2: Railway

**Características:**
- Deploy simplificado (CLI o GitHub)
- Soporte nativo para Docker
- Variables de entorno seguras
- Métricas básicas incluidas

**Costo:**
- $5/mes de crédito gratuito
- Después: ~$10-15/mes
- **Total: ~$15/mes**

**Limitaciones:**
- Similar a Render en capacidad
- Comunidad más pequeña

**Casos de uso:** Desarrollo, staging, prototipos

### Opción 3: AWS ECS (Escalable para Producción)

**Arquitectura:**
```
Route 53 (DNS)
   ↓
CloudFront (CDN + SSL)
   ↓
Application Load Balancer
   ↓
ECS Fargate (2-10 tasks autoscaling)
   ↓
RDS PostgreSQL (Multi-AZ)
```

**Características:**
- Autoscaling horizontal
- Alta disponibilidad (Multi-AZ)
- Monitoreo avanzado (CloudWatch)
- Integración con servicios AWS

**Costo:**
- ECS Fargate (2 tasks): ~$30/mes
- RDS PostgreSQL (t3.micro): ~$20/mes
- ALB: ~$20/mes
- CloudWatch + otros: ~$10/mes
- **Total: ~$80/mes**

**Casos de uso:** Producción con >10,000 requests/día, SLA crítico

## Seguridad

### Autenticación

**API Key en Headers:**
```
X-API-Key: <token-secreto>
```

**Implementación:**
- Validación en cada endpoint
- Tokens diferentes por ambiente (dev/staging/prod)
- Rotación trimestral de tokens
- Registro de accesos por API key

### Rate Limiting

**Límites propuestos:**
```
Por IP:
- Clasificación individual: 100 requests/minuto
- Clasificación batch: 10 requests/minuto
- Endpoints de consulta: 200 requests/minuto
```

**Estrategia:**
- Middleware de FastAPI (slowapi)
- Respuesta HTTP 429 cuando se excede
- Headers informativos (X-RateLimit-Remaining)

### Protección de Datos

**Datos sensibles:**
- Comentarios de clientes (PII potencial)
- Metadata de llamadas

**Medidas:**
- Encriptación en tránsito (HTTPS/TLS)
- Encriptación en reposo (PostgreSQL con pg_crypto)
- Logs anonimizados (hash de IDs)
- Retención de logs: 90 días
- Backup diario con retención 30 días

## Monitoreo y Observabilidad

### Métricas de Negocio

| Métrica | Descripción | Umbral de Alerta |
|---------|-------------|------------------|
| Predicciones/hora | Throughput del sistema | <10/hora (bajo uso) |
| Distribución categorías | Balance de motivos | Cambio >20% en 24h |
| Confianza promedio | Calidad de clasificación | <0.70 |
| Tasa de error | % requests fallidos | >1% |

### Métricas Técnicas

| Métrica | Descripción | Umbral de Alerta |
|---------|-------------|------------------|
| Latencia p95 | Tiempo de respuesta | >500ms |
| Uso de CPU | Carga del servidor | >80% |
| Uso de RAM | Memoria consumida | >90% |
| Disponibilidad | Uptime del servicio | <99.5% |

### Dashboard Propuesto

**Panel 1: Salud del Sistema**
- Uptime (%)
- Requests por minuto (gráfico de línea)
- Latencia p50/p95/p99 (gráfico de área)
- Tasa de error (%)

**Panel 2: Análisis de Negocio**
- Distribución de categorías (gráfico de torta)
- Tendencia de categorías (últimos 7 días, líneas)
- Comentarios por hora del día (heatmap)
- Confianza promedio por categoría (barras)

**Panel 3: Alertas Activas**
- Lista de alertas recientes
- Categorías con incremento anormal
- Latencias fuera de rango

### Alertas Configuradas

**Críticas (notificación inmediata):**
- Servicio caído (downtime >5min)
- Tasa de error >5%
- Latencia p95 >2s

**Advertencias (notificación agregada):**
- CPU >80% por 15min
- Incremento >30% en categoría "Demoras"
- Confianza promedio <0.65

## Integración con Sistemas Existentes

### 1. CRM (Salesforce, Zendesk)

**Webhook en tiempo real:**
```
Cliente reporta queja → API clasifica → Webhook a CRM → Ticket auto-etiquetado

Payload:
{
  "ticket_id": "TKT-12345",
  "categoria": "Falta de solución",
  "confianza": 0.85,
  "prioridad_sugerida": "Alta"
}
```

### 2. Sistema de Tickets Interno

**API call durante creación de ticket:**
```
Formulario web → JavaScript llama API → Muestra categoría sugerida

Usuario puede:
- Aceptar categoría sugerida (1 clic)
- Modificar manualmente
```

### 3. Dashboard de Analytics

**Queries SQL programadas:**
```
Dashboard BI → Query a PostgreSQL → Gráficos de tendencias

Queries:
- Evolución diaria de categorías
- Correlación categoría vs tiempo de resolución
- Top palabras clave por categoría
```

## Mantenimiento y Evolución

### Reentrenamiento del Modelo

**Frecuencia:** Mensual o cuando drift >15%

**Proceso:**
1. Extracción de comentarios nuevos (PostgreSQL)
2. Etiquetado manual de muestra (200 comentarios)
3. Validación de categorías existentes
4. Reentrenamiento de LDA en notebook
5. Evaluación de métricas (perplexity, coherencia)
6. Actualización de modelos en producción
7. A/B testing (7 días)
8. Rollout completo si mejora >5%

**Versionamiento:**
- Modelos versionados: `lda_model_v1.0.joblib`, `lda_model_v1.1.joblib`
- API mantiene versión anterior 30 días
- Endpoint `/health` reporta versión activa

### Monitoreo de Drift

**Indicadores:**
- Distribución de categorías cambia >20%
- Confianza promedio baja <0.65
- Nuevas palabras clave frecuentes no capturadas

**Acción:**
- Alerta automática a equipo de ciencia de datos
- Análisis de comentarios recientes
- Decisión de reentrenamiento

## Plan de Implementación

| Fase | Actividad | Duración | Responsable |
|------|-----------|----------|-------------|
| 1 | Desarrollo API FastAPI | 3 días | Desarrollador Backend |
| 2 | Containerización (Docker) | 1 día | DevOps |
| 3 | Tests unitarios e integración | 2 días | QA + Desarrollador |
| 4 | Deploy en ambiente staging (Render) | 1 día | DevOps |
| 5 | Pruebas de carga y ajuste | 1 día | QA + DevOps |
| 6 | Configuración de monitoreo | 1 día | DevOps |
| 7 | Documentación técnica | 1 día | Desarrollador |
| 8 | Deploy a producción | 1 día | DevOps |
| 9 | Capacitación a usuarios | 1 día | Líder Técnico |
| **Total** | | **12 días** | |

**Hitos:**
- Día 4: API funcional en staging
- Día 8: Ambiente productivo listo
- Día 12: Liberación a usuarios finales

## Costos Totales

### Año 1 (MVP en Render)

| Concepto | Costo Mensual | Costo Anual |
|----------|---------------|-------------|
| Hosting API (Render) | $7 | $84 |
| Base de datos PostgreSQL | $7 | $84 |
| Monitoreo (Grafana Cloud free tier) | $0 | $0 |
| SSL/DNS (incluido) | $0 | $0 |
| **Subtotal Operación** | **$14** | **$168** |
| Desarrollo inicial (12 días) | - | $4,800 |
| Mantenimiento (4h/mes) | $200 | $2,400 |
| **Total Año 1** | | **$7,368** |

### Año 2+ (Escalado a AWS)

| Concepto | Costo Mensual | Costo Anual |
|----------|---------------|-------------|
| ECS Fargate (2 tasks) | $30 | $360 |
| RDS PostgreSQL | $20 | $240 |
| ALB + networking | $25 | $300 |
| CloudWatch + logs | $10 | $120 |
| **Subtotal Operación** | **$85** | **$1,020** |
| Mantenimiento (6h/mes) | $300 | $3,600 |
| **Total Año 2+** | | **$4,620** |

**ROI Estimado:**
- Reducción de 40% en tiempo de primera respuesta
- Ahorro de 20h/mes en routing manual de tickets
- Valor: $1,000/mes × 12 = $12,000/año
- **ROI: 63% en Año 1, 160% en Año 2+**

## Riesgos y Mitigaciones

| Riesgo | Probabilidad | Impacto | Mitigación |
|--------|--------------|---------|------------|
| Drift del modelo (categorías cambian) | Media | Alto | Monitoreo continuo, reentrenamiento mensual |
| Spike de tráfico inesperado | Baja | Medio | Rate limiting, autoscaling en AWS |
| Falla del servicio cloud | Baja | Alto | Backup en segundo provider, SLA 99.9% |
| Clasificación incorrecta crítica | Media | Medio | Confianza mínima 0.6, revisión humana casos <0.7 |
| Exposición de API sin autenticación | Baja | Alto | API key obligatoria, auditoría de accesos |

## Conclusiones

### Beneficios del Sistema

1. **Automatización:** Clasificación de 100% de comentarios sin intervención manual
2. **Rapidez:** Latencia <100ms por comentario (clasificación en tiempo real)
3. **Escalabilidad:** Desde 100 hasta 100,000 comentarios/día con ajustes de infraestructura
4. **Trazabilidad:** Log completo de clasificaciones para auditoría y mejora continua
5. **Bajo costo:** Inicio con $14/mes, ROI positivo desde mes 1

### Recomendaciones

**Para MVP (primeros 3 meses):**
- Desplegar en Render (costo/beneficio óptimo)
- Monitoreo básico con métricas nativas
- Integración con 1-2 sistemas (CRM + tickets)

**Para escalar (6-12 meses):**
- Migrar a AWS ECS con autoscaling
- Implementar Prometheus + Grafana
- A/B testing de nuevas versiones del modelo

**Próximos pasos inmediatos:**
1. Aprobar arquitectura propuesta
2. Asignar equipo de desarrollo (1 backend + 1 devops)
3. Provisionar ambiente de staging
4. Iniciar desarrollo sprint de 2 semanas

El sistema propuesto permite una implementación incremental con bajo riesgo, comenzando con una solución simple y escalando según demanda real.